In [ ]:
pip install session_info

In [ ]:
import session_info

In [ ]:
pip install git+https://github.com/theislab/scgen.git

In [ ]:
pip install scvi-tools==1.1.1

In [ ]:
pip install lamindb

In [ ]:
pip install tiledbsoma

In [ ]:
import scanpy as sc
import torch
import logging
import scgen # Development version only works!!!!!!!!! Confirmed 25
import sklearn
import seaborn as sns
import torch
import warnings
import os
import sys
import re





#import numpy as np

# Remember to downgrade scvi-tools (Sometimes need to downgrade not always, use pip install scvi-tools 1.6, 1.1.1 ) 
# sqrt issue in latent space

# 1. Download scanpy
# 2. Download scgen (not development version) --use that one for now

In [ ]:
session_info.show()

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
adata = sc.read("/work/scGen_Human_vascular/new_data_fix_may/healthy_hamstring_processed_adata_raw.h5ad")

In [ ]:
adata.obs["donor_id"]

In [ ]:
# Split the data set into train and test
from sklearn.model_selection import train_test_split


split_key = "split"
adata.obs[split_key] = "train"
idx = list(range(len(adata)))
idx_train, idx_test = train_test_split(adata.obs_names, test_size=0.1, random_state=42)
adata.obs.loc[idx_train, split_key] = "train"
adata.obs.loc[idx_test, split_key] = "test"

# Filter the data to use only the training set and make a copy
adata_train = adata[adata.obs[split_key] == "train"].copy()
adata_test = adata[adata.obs[split_key] == "test"].copy()

In [ ]:
scgen.SCGEN.setup_anndata(adata_train, batch_key="donor_id", labels_key="cell_type")

In [ ]:
model = scgen.SCGEN(adata_train)
model.save("work/scGen_Human_vascular_new_run_fix/batch_removal_runs/HH_batch", overwrite=True)

In [ ]:
model

In [ ]:
model.train(
    max_epochs=300,
    batch_size=32,
    early_stopping=True,
    early_stopping_patience=100,
)

In [ ]:
torch.load("/work/scGen_Human_vascular/work/scGen_Human_vascular_GPU_trained/saved_models/scGen_HH_GPU_run/model.pt", adata = adata_train, weights_only=False)

In [ ]:
model.load(
    "/work/scGen_Human_vascular/work/scGen_Human_vascular_GPU_trained/saved_models/scGen_HH_GPU_run",
    adata=adata_train
)


In [ ]:
# Batch removal
corrected_adata = model.batch_removal()
corrected_adata

In [ ]:
pred, delta = model.predict(ctrl_key="MSK0782",
                            stim_key="MSK0782",
                            adata_to_predict=adata_test
                           )

In [ ]:
# If rec is an AnnData object, extract the X attribute (i.e., the data matrix)
import anndata
if isinstance(pred, anndata.AnnData):
    pred = pred.X

# Now, rec should be a numpy array or sparse matrix, which is what obsm expects
adata_test.obsm["X_reconstructed"] = pred

# Save the entire object with the reconstructed data
adata_test.write("scGen_HH_raw_fix_adata_post_with_latent_and_recon_batch_removed.h5ad")